# Advent of Code 2016
#### Author: Jeroen van Erp
#### Date: 2016-12-29

This year, like last year I've played in the Advent of Code. Together with a few colleagues we've had a nice internal competition. You can find some of my colleagues results at:

- Serge Beaumont: https://github.com/sbeaumont/AoC
- Bastiaan Bakker: https://github.com/bastiaanb/advent-of-code-2016
- Frank van Wijk: https://github.com/fvanwijk/adventofcode-2016
- Arnout Engelen: https://github.com/raboof/adventofcode
- Jasper Stein: https://github.com/jasperstein/idris-experiments/tree/aoc2016

## Preamble
First let's set up some common functions and imports

In [1]:
import re
from heapq import heappush, heappop
from collections import namedtuple, defaultdict, Counter
from hashlib import md5
from bitarray import bitarray
import math
from functools import partial, lru_cache
from itertools import permutations, combinations, islice
import numpy as np
import networkx as nx
import urllib

def Input(day):
    "Open this day's input file."
    filename = 'day{}.in'.format(day)
    try:
        return open(filename)
    except FileNotFoundError:
        return urllib.request.urlopen("https://raw.githubusercontent.com/hierynomus/code-challenges/master/2016-adventofcode.com/" + filename).text

    
def neighbours4(point):
    """All horizontal and vertical neighbours of a point in a grid"""
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        yield (point[0] + dx, point[1] + dy)

def neighbours8(point):
    """All neighbours (including diagonals) of a point in a grid"""
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 and dy != 0:
                yield (point[0] + dx, point[1] + dy)
                
def manhattan_distance(p1, p2):
    """Return the Manhattan distance between two points"""
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def window(seq, window_size):
    """Generator that slides a window over the iterable"""
    for i in range(len(seq) - window_size + 1):
        yield seq[i:i + window_size]
        
def astar(initial_state, f_next, f_heuristic):
    """Do an A* search"""
    frontier = [(f_heuristic(initial_state), 0, initial_state)]
    seen_states = set([initial_state])
    previous_states = {initial_state: None}
    while frontier:
        cost, path_length, state = heappop(frontier)
        if f_heuristic(state) == 0:
            yield path(state, previous_states)
        else:
            for next_state in f_next(state):
                # Do we need to check whether the path_cost is lower than previously seen?
                # This is guaranteed if the heuristic is consistent and only depends on the next_state
                # I.e. this is not a weighted graph!
                # Else we need to implement reprioritization
                if next_state in seen_states:
                    continue
                heappush(frontier, (path_length + f_heuristic(next_state), path_length + 1, next_state))
                seen_states.add(next_state)
                previous_states[next_state] = state
    return "FAIL!"


def path(state, previous_states):
    while state:
        yield state
        state = previous_states[state]


def path_length(path):
    pl = 0
    for s in path:
        pl += 1
    return pl

## Day 1: No Time for a Taxicab

### Part 1
Santa's sleigh uses a very high-precision clock to guide its movements, and the clock's oscillator is regulated by stars. Unfortunately, the stars have been stolen... by the Easter Bunny. To save Christmas, Santa needs you to retrieve all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day in the advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

You're airdropped near **Easter Bunny Headquarters** in a city somewhere. "Near", unfortunately, is as close as you can get - the instructions on the Easter Bunny Recruiting Document the Elves intercepted start here, and nobody had time to work them out further.

The Document indicates that you should start at the given coordinates (where you just landed) and face North. Then, follow the provided sequence: either turn left (L) or right (R) 90 degrees, then walk forward the given number of blocks, ending at a new intersection.

There's no time to follow such ridiculous instructions on foot, though, so you take a moment and work out the destination. Given that you can only walk on the street grid of the city, how far is the shortest path to the destination?

For example:

- Following `R2, L3` leaves you 2 blocks East and 3 blocks North, or 5 blocks away.
- `R2, R2, R2` leaves you 2 blocks due South of your starting position, which is 2 blocks away.
- `R5, L5, R5, R3` leaves you 12 blocks away.

**How many blocks away** is Easter Bunny HQ?

In [2]:
# 0 = North, 1 = West, 2 = South, 3 = East
dirs = {0: (0, 1), 1: (1, 0), 2: (0, -1), 3: (-1, 0)}


def walk(point, direction, instruction):
    x, y = point
    direction = turn(direction, instruction[0])
    dx, dy = dirs[direction]
    for _ in range(int(instruction[1:])):
        x, y = x + dx, y + dy
        yield (x, y), direction

def turn(direction, c):
    return (direction + (1 if c == 'R' else -1)) % 4

with Input(1) as f:
    direction = 0
    position = (0, 0)
    for instruction in f.readline().split(', '):
        position, direction = list(walk(position, direction, instruction))[-1]

    print("Day 1.1: {}".format(manhattan_distance((0, 0), position)))

Day 1.1: 287


### Part 2
Then, you notice the instructions continue on the back of the Recruiting Document. Easter Bunny HQ is actually at the first location you visit twice.

For example, if your instructions are `R8, R4, R4, R8`, the first location you visit twice is 4 blocks away, due East.

How many blocks away is the **first location you visit twice**?

In [3]:
with Input(1) as f:
    def solve():
        direction = 0
        position = (0, 0)
        visited = set()
        for instruction in f.readline().split(', '):
            for p, d in walk(position, direction, instruction):
                if p in visited:
                    yield p
                visited.add(p)
                position, direction = p, d
    print("Day 1.2: {}".format(manhattan_distance((0, 0), next(solve()))))


Day 1.2: 133


## Day 2: Bathroom security
You arrive at **Easter Bunny Headquarters** under cover of darkness. However, you left in such a rush that you forgot to use the bathroom! Fancy office buildings like this one usually have keypad locks on their bathrooms, so you search the front desk for the code.

"In order to improve security," the document you find says, "bathroom codes will no longer be written down. Instead, please memorize and follow the procedure below to access the bathrooms."

The document goes on to explain that each button to be pressed can be found by starting on the previous button and moving to adjacent buttons on the keypad: U moves up, D moves down, L moves left, and R moves right. Each line of instructions corresponds to one button, starting at the previous button (or, for the first line, **the "5" button**); press whatever button you're on at the end of each line. If a move doesn't lead to a button, ignore it.

You can't hold it much longer, so you decide to figure out the code as you walk to the bathroom. You picture a keypad like this:

```
1 2 3
4 5 6
7 8 9
```
Suppose your instructions are:

```
ULL
RRDDD
LURDL
UUUUD
```

- You start at "5" and move up (to "2"), left (to "1"), and left (you can't, and stay on "1"), so the first button is `1`.
- Starting from the previous button ("1"), you move right twice (to "3") and then down three times (stopping at "9" after two moves and ignoring the third), ending up with `9`.
- Continuing from "9", you move left, up, right, down, and left, ending with `8`.
- Finally, you move up four times (stopping at "2"), then down once, ending with `5`.
So, in this example, the bathroom code is `1985`.

Your puzzle input is the instructions from the document you found at the front desk. What is the `bathroom code`?

In [4]:
keypad = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
moves = {
    'U': lambda k: (k[0], max(0, k[1] - 1)),
    'D': lambda k: (k[0], min(2, k[1] + 1)),
    'L': lambda k: (max(0, k[0] - 1), k[1]),
    'R': lambda k: (min(2, k[0] + 1), k[1])
}

with Input(2) as f:
    key = (1, 1)
    sequence = []
    for line in f:
        for c in line.strip():
            key = moves[c](key)
        sequence.append(keypad[key[1]][key[0]])
    print("Day 2.1: {}".format(''.join(map(str, sequence))))

Day 2.1: 78293


### Part 2
You finally arrive at the bathroom (it's a several minute walk from the lobby so visitors can behold the many fancy conference rooms and water coolers on this floor) and go to punch in the code. Much to your bladder's dismay, the keypad is not at all like you imagined it. Instead, you are confronted with the result of hundreds of man-hours of bathroom-keypad-design meetings:

```
    1
  2 3 4
5 6 7 8 9
  A B C
    D
```
You still start at "5" and stop when you're at an edge, but given the same instructions as above, the outcome is very different:

- You start at "5" and don't move at all (up and left are both edges), ending at `5`.
- Continuing from "5", you move right twice and down three times (through "6", "7", "B", "D", "D"), ending at `D`.
- Then, from "D", you move five more times (through "D", "B", "C", "C", "B"), ending at `B`.
- Finally, after five more moves, you end at `3`.
So, given the actual keypad layout, the code would be `5DB3`.

Using the same instructions in your puzzle input, what is the correct **bathroom code**?

In [5]:
keypad = {
    '1': {'D': '3'},
    '2': {'R': '3', 'D': '6'},
    '3': {'U': '1', 'L': '2', 'D': '7', 'R': '4'},
    '4': {'L': '3', 'D': '8'},
    '5': {'R': '6'},
    '6': {'L': '5', 'U': '2', 'R': '7', 'D': 'A'},
    '7': {'L': '6', 'U': '3', 'R': '8', 'D': 'B'},
    '8': {'L': '7', 'U': '4', 'R': '9', 'D': 'C'},
    '9': {'L': '8'},
    'A': {'U': '6', 'R': 'B'},
    'B': {'L': 'A', 'U': '7', 'R': 'C', 'D': 'D'},
    'C': {'U': '8', 'L': 'B'},
    'D': {'U': 'A'}
}

with Input(2) as f:
    key = '5'
    sequence = []
    for line in f:
        for c in line.strip():
            key = keypad[key][c] if c in keypad[key] else key
        sequence.append(key)
    print("Day 2.2: {}".format(''.join(sequence)))

Day 2.2: AC8C8


## Day 3: Squares With Three Sides
Now that you can think clearly, you move deeper into the labyrinth of hallways and office furniture that makes up this part of Easter Bunny HQ. This must be a graphic design department; the walls are covered in specifications for triangles.

Or are they?

The design document gives the side lengths of each triangle it describes, but... `5 10 25`? Some of these aren't triangles. You can't help but mark the impossible ones.

In a valid triangle, the sum of any two sides must be larger than the remaining side. For example, the "triangle" given above is impossible, because `5 + 10` is not larger than `25`.

In your puzzle input, **how many** of the listed triangles are **possible**?

In [6]:
def is_triangle(triangle):
    x, y, z = sorted(triangle)
    return x + y > z

with Input(3) as f:
    triangles = [[int(c) for c in line.strip().split()] for line in f]
    print("Day 3.1: {}".format(sum(map(is_triangle, triangles))))

Day 3.1: 869


Now that you've helpfully marked up their design documents, it occurs to you that triangles are specified in groups of three **vertically**. Each set of three numbers in a column specifies a triangle. Rows are unrelated.

For example, given the following specification, numbers with the same hundreds digit would be part of the same triangle:

```
101 301 501
102 302 502
103 303 503
201 401 601
202 402 602
203 403 603
```
In your puzzle input, and instead reading by columns, **how many** of the listed triangles are **possible**?

In [7]:
triangles = np.transpose(np.array(triangles)).reshape(len(triangles), 3)
print("Day 3.2: {}".format(sum(map(is_triangle, triangles))))

Day 3.2: 1544


## Day 4: Security Through Obscurity
Finally, you come across an information kiosk with a list of rooms. Of course, the list is encrypted and full of decoy data, but the instructions to decode the list are barely hidden nearby. Better remove the decoy data first.

Each room consists of an encrypted name (lowercase letters separated by dashes) followed by a dash, a sector ID, and a checksum in square brackets.

A room is real (not a decoy) if the checksum is the five most common letters in the encrypted name, in order, with ties broken by alphabetization. For example:

- `aaaaa-bbb-z-y-x-123[abxyz]` is a real room because the most common letters are `a` (5), `b` (3), and then a tie between `x`, `y`, and `z`, which are listed alphabetically.
- `a-b-c-d-e-f-g-h-987[abcde]` is a real room because although the letters are all tied (1 of each), the first five are listed alphabetically.
- `not-a-real-room-404[oarel]` is a real room.
- `totally-real-room-200[decoy]` is not.

Of the real rooms from the list above, the sum of their sector IDs is `1514`.

What is the **sum of the sector IDs of the real rooms**?

In [8]:
room_re = re.compile("(?P<name>[a-z-]+)-(?P<sector>[0-9]+)\[(?P<checksum>[a-z]{5})\]")

def real_room(room):
    counter = Counter(room['name'])
    del counter['-']
    top5 = sorted([(-n, c) for c, n in counter.most_common()])[:5]
    top = ''.join([c for n, c in top5])
    return room['checksum'] == top

with Input(4) as f:
    rooms = list(filter(real_room, [room_re.search(r).groupdict() for r in f]))
    print("Day 4.1: {}".format(sum(map(lambda r: int(r['sector']), rooms))))

Day 4.1: 409147


### Part 2
With all the decoy data out of the way, it's time to decrypt this list and get moving.

The room names are encrypted by a state-of-the-art shift cipher, which is nearly unbreakable without the right software. However, the information kiosk designers at Easter Bunny HQ were not expecting to deal with a master cryptographer like yourself.

To decrypt a room name, rotate each letter forward through the alphabet a number of times equal to the room's sector ID. `A` becomes `B`, `B` becomes `C`, `Z` becomes `A`, and so on. Dashes become spaces.

For example, the real name for `qzmt-zixmtkozy-ivhz-343` is `very encrypted name`.

**What is the sector ID** of the room where North Pole objects are stored?

In [9]:
def solve(rooms):
    for room in rooms:
        name, sector = room['name'], int(room['sector'])
        decrypted = ''.join([' ' if c == '-' else chr((((ord(c) - 97) + sector) % 26) + 97) for c in name])
        if 'northpole' in decrypted:
            yield sector

print("Day 4.2: {}".format(next(solve(rooms))))

Day 4.2: 991


## Day 5: How About a Nice Game of Chess
You are faced with a security door designed by Easter Bunny engineers that seem to have acquired most of their security knowledge by watching hacking movies.

The **eight-character password** for the door is generated one character at a time by finding the MD5 hash of some Door ID (your puzzle input) and an increasing integer index (starting with `0`).

A hash indicates the **next character** in the password if its hexadecimal representation starts with **five zeroes**. If it does, the sixth character in the hash is the next character of the password.

For example, if the Door ID is `abc`:

- The first index which produces a hash that starts with five zeroes is `3231929`, which we find by hashing `abc3231929`; the sixth character of the hash, and thus the first character of the password, is `1`.
- `5017308` produces the next interesting hash, which starts with `000008f82...`, so the second character of the password is `8`.
- The third time a hash starts with five zeroes is for `abc5278568`, discovering the character `f`.

In this example, after continuing this search a total of eight times, the password is `18f47a30`.

Given the actual Door ID, **what is the password**?

Your puzzle input is `reyedfim`

In [10]:
def door_password(inp):
    hasher = md5(inp.encode('utf-8'))
    idx = 0
    while True:
        copy = hasher.copy()
        copy.update(str(idx).encode('utf-8'))
        digest = copy.hexdigest()
        idx += 1
        if digest[:5] == '00000':
            yield digest

print("Day 5.1: {}".format(''.join([c[5] for _, c in zip(range(8), door_password('reyedfim'))])))


Day 5.1: f97c354d


### Part 2
As the door slides open, you are presented with a second door that uses a slightly more inspired security mechanism. Clearly unimpressed by the last version (in what movie is the password decrypted **in order**?!), the Easter Bunny engineers have worked out a better solution.

Instead of simply filling in the password from left to right, the hash now also indicates the **position** within the password to fill. You still look for hashes that begin with five zeroes; however, now, the **sixth** character represents the **position** (`0`-`7`), and the **seventh** character is the character to put in that position.

A hash result of `000001f` means that `f` is the **second** character in the password. Use only the **first result** for each position, and ignore invalid positions.

For example, if the Door ID is `abc`:

- The first interesting hash is from `abc3231929`, which produces `0000015...`; so, `5` goes in position `1`: `_5______`.
- In the previous method, `5017308` produced an interesting hash; however, it is ignored, because it specifies an invalid position (`8`).
- The second interesting hash is at index `5357525`, which produces `000004e...`; so, `e` goes in position `4`: `_5__e___`.

You almost choke on your popcorn as the final character falls into place, producing the password `05ace8e3`.

Given the actual Door ID and this new method, **what is the password**? Be extra proud of your solution if it uses a cinematic "decrypting" animation.

In [11]:
password = [None] * 8
generator = door_password('reyedfim')
while None in password:
    h = next(generator)
    pos = int(h[5], 16)
    if pos < 8 and not password[pos]:
        password[pos] = h[6]
        
print("Day 5.2: {}".format(''.join(password)))

Day 5.2: 863dde27


## Day 6: Signals and Noise
Something is jamming your communications with Santa. Fortunately, your signal is only partially jammed, and protocol in situations like this is to switch to a simple repetition code to get the message through.

In this model, the same message is sent repeatedly. You've recorded the repeating message signal (your puzzle input), but the data seems quite corrupted - almost too badly to recover. **Almost**.

All you need to do is figure out which character is most frequent for each position. For example, suppose you had recorded the following messages:

```
eedadn
drvtee
eandsr
raavrd
atevrs
tsrnev
sdttsa
rasrtv
nssdts
ntnada
svetve
tesnvt
vntsnd
vrdear
dvrsen
enarar
```

The most common character in the first column is `e`; in the second, `a`; in the third, `s`, and so on. Combining these characters returns the error-corrected message, easter.

Given the recording in your puzzle input, **what is the error-corrected version** of the message being sent?

In [12]:
with Input(6) as f:
    inp = np.array([list(line.strip()) for line in f])
    counters = [Counter(p).most_common() for p in np.transpose(inp)]

print("Day 6.1: {}".format(''.join([c[0][0] for c in counters])))

Day 6.1: zcreqgiv


### Part 2
Of course, that **would** be the message - if you hadn't agreed to use a **modified repetition code** instead.

In this modified code, the sender instead transmits what looks like random data, but for each character, the character they actually want to send is **slightly less likely** than the others. Even after signal-jamming noise, you can look at the letter distributions in each column and choose the **least common** letter to reconstruct the original message.

In the above example, the least common character in the first column is `a`; in the second, `d`, and so on. Repeating this process for the remaining characters produces the original message, `advent`.

Given the recording in your puzzle input and this new decoding methodology, **what is the original message** that Santa is trying to send?

In [13]:
print("Day 6.2: {}".format(''.join(c[-1][0] for c in counters)))

Day 6.2: pljvorrk


## Day 7: Internet Protocol Version 7
While snooping around the local network of EBHQ, you compile a list of IP addresses (they're IPv7, of course; IPv6 is much too limited). You'd like to figure out which IPs support **TLS** (transport-layer snooping).

An IP supports TLS if it has an Autonomous Bridge Bypass Annotation, or **ABBA**. An ABBA is any four-character sequence which consists of a pair of two different characters followed by the reverse of that pair, such as xyyx or abba. However, the IP also must not have an ABBA within any hypernet sequences, which are contained by **square brackets**.

For example:

- `abba[mnop]qrst` supports TLS (`abba` outside square brackets).
- `abcd[bddb]xyyx` does not support TLS (`bddb` is within square brackets, even though `xyyx` is outside square brackets).
- `aaaa[qwer]tyui` does not support TLS (`aaaa` is invalid; the interior characters must be different).
- `ioxxoj[asdfgh]zxcvbn` supports TLS (`oxxo` is outside square brackets, even though it's within a larger string).

**How many IPs** in your puzzle input support TLS?

In [14]:
def split_ip(ip):
    """Split the IP into a string of comma separated supernets and a string of comma-separated hypernets"""
    parts = re.split('\[|\]', ip)
    return ','.join(parts[::2]), ','.join(parts[1::2])

def abba(net):
    return any([a == d and b == c and a != b for a, b, c, d in window(net, 4)])

def tls(split_ip): return abba(split_ip[0]) and not abba(split_ip[1])

with Input(7) as f:
    splitips = [split_ip(ip) for ip in f]

print("Day 7.1: {}".format(sum(map(tls, splitips))))

Day 7.1: 115


### Part 2
You would also like to know which IPs support **SSL** (super-secret listening).

An IP supports SSL if it has an Area-Broadcast Accessor, or **ABA**, anywhere in the supernet sequences (outside any square bracketed sections), and a corresponding Byte Allocation Block, or **BAB**, anywhere in the hypernet sequences. An ABA is any three-character sequence which consists of the same character twice with a different character between them, such as `xyx` or `aba`. A corresponding BAB is the same characters but in reversed positions: `yxy` and `bab`, respectively.

For example:

- `aba[bab]xyz` supports SSL (`aba` outside square brackets with corresponding `bab` within square brackets).
- `xyx[xyx]xyx` does not support SSL (`xyx`, but no corresponding `yxy`).
- `aaa[kek]eke` supports SSL (`eke` in supernet with corresponding `kek` in hypernet; the `aaa` sequence is not related, because the interior character must be different).
- `zazbz[bzb]cdb` supports SSL (`zaz` has no corresponding `aza`, but `zbz` has a corresponding `bzb`, even though `zaz` and `zbz` overlap).

**How many IPs** in your puzzle input support SSL?

In [15]:
def aba(net):
    for a, b, c in window(net, 3):
        if a == c and a != b:
            yield [a, b, a]

def ssl(split_ip): return any([''.join([b, a, b]) in split_ip[1] for a, b, _ in aba(split_ip[0])])

print("Day 7.2: {}".format(sum(map(ssl, splitips))))

Day 7.2: 231


## Day 8: Two-Factor Authentication
You come across a door implementing what you can only assume is an implementation of two-factor authentication after a long game of requirements telephone.

To get past the door, you first swipe a keycard (no problem; there was one on a nearby desk). Then, it displays a code on a little screen, and you type that code on a keypad. Then, presumably, the door unlocks.

Unfortunately, the screen has been smashed. After a few minutes, you've taken everything apart and figured out how it works. Now you just have to work out what the screen **would** have displayed.

The magnetic strip on the card you swiped encodes a series of instructions for the screen; these instructions are your puzzle input. The screen is **`50` pixels wide and `6` pixels tall**, all of which start **off**, and is capable of three somewhat peculiar operations:

- `rect AxB` turns **on** all of the pixels in a rectangle at the top-left of the screen which is `A` wide and `B` tall.
- `rotate row y=A by B` shifts all of the pixels in row `A` (`0` is the top row) **right** by `B` pixels. Pixels that would fall off the right end appear at the left end of the row.
- `rotate column x=A by B` shifts all of the pixels in column `A` (`0` is the left column) **down** by `B` pixels. Pixels that would fall off the bottom appear at the top of the column.

For example, here is a simple sequence on a smaller screen:

- `rect 3x2` creates a small rectangle in the top-left corner:
```
###....
###....
.......
```

- `rotate column x=1 by 1` rotates the second column down by one pixel:
```
#.#....
###....
.#.....
```

- `rotate row y=0 by 4` rotates the top row right by four pixels:
```
....#.#
###....
.#.....
```
- `rotate column x=1 by 1` again rotates the second column down by one pixel, causing the bottom pixel to wrap back to the top:
```
.#..#.#
#.#....
.#.....
```

As you can see, this display technology is extremely powerful, and will soon dominate the tiny-code-displaying-screen market. That's what the advertisement on the back of the display tries to convince you, anyway.

There seems to be an intermediate check of the voltage used by the display: after you swipe your card, if the screen did work, **how many pixels should be lit**?

In [16]:
class Display(object):
    def __init__(self, display=np.array([[' '] * 50] * 6)):
        self.display = display

    def rect(self, x, y):
        d = self.display.copy()
        d[:y, :x] = chr(9608)
        return Display(d)

    def show(self):
        return '\n'.join([''.join(l) for l in self.display])

    def rotate_row(self, y, pos):
        d = self.display.copy()
        d[y] = np.roll(d[y], pos)
        return Display(d)

    def rotate_col(self, x, pos):
        d = self.display.copy()
        d = np.transpose(d)
        d[x] = np.roll(d[x], pos)
        return Display(np.transpose(d))

    def count(self):
        return (self.display == chr(9608)).sum()

with Input(8) as f:
    display = Display()
    for instruction in f:
        if 'rect' in instruction:
            a, b = map(int, instruction[5:].split('x'))
            display = display.rect(a, b)
        if 'rotate row' in instruction:
            i = instruction.split()
            y, pos = int(i[2].split('=')[1]), int(i[4])
            display = display.rotate_row(y, pos)
        if 'rotate col' in instruction:
            i = instruction.split()
            x, pos = int(i[2].split('=')[1]), int(i[4])
            display = display.rotate_col(x, pos)

print("Day 8.1: {}".format(display.count()))

Day 8.1: 121


### Part 2
You notice that the screen is only capable of displaying capital letters; in the font it uses, each letter is `5` pixels wide and `6` tall.

After you swipe your card, **what code is the screen trying to display**?

In [17]:
print("Day 8.2:\n{}".format(display.show()))

Day 8.2:
███  █  █ ███  █  █  ██  ████  ██  ████  ███ █    
█  █ █  █ █  █ █  █ █  █ █    █  █ █      █  █    
█  █ █  █ █  █ █  █ █    ███  █  █ ███    █  █    
███  █  █ ███  █  █ █    █    █  █ █      █  █    
█ █  █  █ █ █  █  █ █  █ █    █  █ █      █  █    
█  █  ██  █  █  ██   ██  ████  ██  ████  ███ ████ 


The solution reads: `RURUCEOEIL`

## Day 9: Explosives in Cyperspace
Wandering around a secure area, you come across a datalink port to a new part of the network. After briefly scanning it for interesting files, you find one file in particular that catches your attention. It's compressed with an experimental format, but fortunately, the documentation for the format is nearby.

The format compresses a sequence of characters. Whitespace is ignored. To indicate that some sequence should be repeated, a marker is added to the file, like `(10x2)`. To decompress this marker, take the subsequent `10` characters and repeat them `2` times. Then, continue reading the file **after** the repeated data. The marker itself is not included in the decompressed output.

If parentheses or other characters appear within the data referenced by a marker, that's okay - treat it like normal data, not a marker, and then resume looking for markers after the decompressed section.

For example:

- `ADVENT` contains no markers and decompresses to itself with no changes, resulting in a decompressed length of `6`.
- `A(1x5)BC` repeats only the `B` a total of `5` times, becoming `ABBBBBC` for a decompressed length of `7`.
- `(3x3)XYZ` becomes `XYZXYZXYZ` for a decompressed length of `9`.
- `A(2x2)BCD(2x2)EFG` doubles the `BC` and `EF`, becoming `ABCBCDEFEFG` for a decompressed length of `11`.
- `(6x1)(1x3)A` simply becomes `(1x3)A` - the `(1x3)` looks like a marker, but because it's within a data section of another marker, it is not treated any differently from the `A` that comes after it. It has a decompressed length of `6`.
- `X(8x2)(3x3)ABCY` becomes `X(3x3)ABC(3x3)ABCY` (for a decompressed length of `18`), because the decompressed data from the `(8x2)` marker (the `(3x3)ABC`) is skipped and not processed further.

What is the **decompressed length** of the file (your puzzle input)? Don't count whitespace.

In [18]:
def decompressed_length(line):
    start, rest = line.split("(", 1) if "(" in line else (line, None)
    length = len(start)
    if not rest:
        return length
    compression, rest = rest.split(")", 1)
    chars, repeat = map(int, compression.split("x"))
    length += chars * repeat
    return length + decompressed_length(rest[chars:])

with Input(9) as f:
    print("Day 9.1: {}".format(decompressed_length(f.read().strip())))

Day 9.1: 115118


### Part 2
Apparently, the file actually uses **version two** of the format.

In version two, the only difference is that markers within decompressed data **are** decompressed. This, the documentation explains, provides much more substantial compression capabilities, allowing many-gigabyte files to be stored in only a few kilobytes.

For example:

- `(3x3)XYZ` still becomes `XYZXYZXYZ`, as the decompressed section contains no markers.
- `X(8x2)(3x3)ABCY` becomes `XABCABCABCABCABCABCY`, because the decompressed data from the `(8x2)` marker is then further decompressed, thus triggering the `(3x3)` marker twice for a total of six `ABC` sequences.
- `(27x12)(20x12)(13x14)(7x10)(1x12)A` decompresses into a string of `A` repeated `241920` times.
- `(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN` becomes `445` characters long.

Unfortunately, the computer you brought probably doesn't have enough memory to actually decompress the file; you'll have to **come up with another way** to get its decompressed length.

What is the **decompressed length** of the file using this improved format?

In [19]:
def decompressed_length(line):
    start, rest = line.split("(", 1) if "(" in line else (line, None)
    length = len(start)
    if not rest:
        return length
    compression, rest = rest.split(")", 1)
    chars, repeat = map(int, compression.split("x"))
    length += decompressed_length(rest[:chars]) * repeat
    return length + decompressed_length(rest[chars:])

with Input(9) as f:
    print("Day 9.2: {}".format(decompressed_length(f.read().strip())))

Day 9.2: 11107527530


## Day 10: Balance Bots
You come upon a factory in which many robots are zooming around handing small microchips to each other.

Upon closer examination, you notice that each bot only proceeds when it has **two** microchips, and once it does, it gives each one to a different bot or puts it in a marked "output" bin. Sometimes, bots take microchips from "input" bins, too.

Inspecting one of the microchips, it seems like they each contain a single number; the bots must use some logic to decide what to do with each chip. You access the local control computer and download the bots' instructions (your puzzle input).

Some of the instructions specify that a specific-valued microchip should be given to a specific bot; the rest of the instructions indicate what a given bot should do with its **lower-value** or **higher-value** chip.

For example, consider the following instructions:

```
value 5 goes to bot 2
bot 2 gives low to bot 1 and high to bot 0
value 3 goes to bot 1
bot 1 gives low to output 1 and high to bot 0
bot 0 gives low to output 2 and high to output 0
value 2 goes to bot 2
```

- Initially, bot `1` starts with a value-`3` chip, and bot `2` starts with a value-`2` chip and a value-`5` chip.
- Because bot `2` has two microchips, it gives its lower one (`2`) to bot `1` and its higher one (`5`) to bot `0`.
- Then, bot `1` has two microchips; it puts the value-`2` chip in output `1` and gives the value-`3` chip to bot `0`.
- Finally, bot `0` has two microchips; it puts the `3` in output `2` and the `5` in output `0`.

In the end, output bin `0` contains a value-`5` microchip, output bin `1` contains a value-`2` microchip, and output bin `2` contains a value-`3` microchip. In this configuration, bot number `2` is responsible for comparing value-`5` microchips with value-`2` microchips.

Based on your instructions, what is the number of the bot that is responsible for comparing value-`61` microchips with value-`17` microchips?

In [20]:
# Holder variable for finding the bot that compares '17' with '61'
bot_part_1 = None

class Output(object):
    """An output bin"""

    def __init__(self, idx):
        self.chips = []
        self.idx = idx

    def receive_chip(self, value):
        self.chips.append(value)


class Bot(object):
    """A Bot ;-)"""
    
    def __init__(self, idx):
        self.chips = []
        self.rule = None
        self.idx = idx
        pass

    def receive_chip(self, value):
        global bot_part_1
        self.chips.append(value)
        self.chips.sort()
        if 17 in self.chips and 61 in self.chips and not bot_part_1:
            bot_part_1 = self.idx
        if len(self.chips) == 2:
            self.process()

    def assign_rule(self, lower, higher):
        self.rule = (lower, higher)
        if len(self.chips) == 2:
            self.process()

    def process(self):
        if self.rule:
            self.rule[0].receive_chip(self.chips[0])
            self.rule[1].receive_chip(self.chips[1])

bots = {}
outputs = {}


def get_actor(actor_idx, actor_type):
    if actor_type == "bot":
        if actor_idx not in bots:
            bots[actor_idx] = Bot(actor_idx)
        return bots[actor_idx]
    else:
        if actor_idx not in outputs:
            outputs[actor_idx] = Output(actor_idx)
        return outputs[actor_idx]


with Input(10) as f:
    for l in f:
        line = l.strip().split()
        if line[0] == "value":
            bot_nr, chip = map(int, (line[5], line[1]))
            get_actor(bot_nr, line[4]).receive_chip(chip)
        if line[0] == "bot":
            get_actor(int(line[1]), line[0]).assign_rule(get_actor(int(line[6]), line[5]), get_actor(int(line[11]), line[10]))

print("Day 10.1: {}".format(bot_part_1))

Day 10.1: 56


### Part 2
What do you get if you **multiply together the values** of one chip in each of outputs `0`, `1`, and `2`?

In [21]:
print("Day 10.2: {}".format(outputs[0].chips[0] * outputs[1].chips[0] * outputs[2].chips[0]))

Day 10.2: 7847


## Day 11: Radioisotope Thermoelectric Generators
You come upon a column of four floors that have been entirely sealed off from the rest of the building except for a small dedicated lobby. There are some radiation warnings and a big sign which reads "Radioisotope Testing Facility".

According to the project status board, this facility is currently being used to experiment with Radioisotope Thermoelectric Generators (RTGs, or simply "generators") that are designed to be paired with specially-constructed microchips. Basically, an RTG is a highly radioactive rock that generates electricity through heat.

The experimental RTGs have poor radiation containment, so they're dangerously radioactive. The chips are prototypes and don't have normal radiation shielding, but they do have the ability to **generate an electromagnetic radiation shield when powered**. Unfortunately, they can only be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.

In other words, if a chip is ever left in the same area as another RTG, and it's not connected to its own RTG, the chip will be **fried**. Therefore, it is assumed that you will follow procedure and keep chips connected to their corresponding RTG when they're in the same room, and away from other RTGs otherwise.

These microchips sound very interesting and useful to your current activities, and you'd like to try to retrieve them. The fourth floor of the facility has an assembling machine which can make a self-contained, shielded computer for you to take with you - that is, if you can bring it all of the RTGs and microchips.

Within the radiation-shielded part of the facility (in which it's safe to have these pre-assembly RTGs), there is an elevator that can move between the four floors. Its capacity rating means it can carry at most yourself and two RTGs or microchips in any combination. (They're rigged to some heavy diagnostic equipment - the assembling machine will detach it for you.) As a security measure, the elevator will only function if it contains at least one RTG or microchip. The elevator always stops on each floor to recharge, and this takes long enough that the items within it and the items on that floor can irradiate each other. (You can prevent this if a Microchip and its Generator end up on the same floor in this way, as they can be connected while the elevator is recharging.)

You make some notes of the locations of each component of interest (your puzzle input). Before you don a hazmat suit and start moving things around, you'd like to have an idea of what you need to do.

When you enter the containment area, you and the elevator will start on the first floor.

For example, suppose the isolated area has the following arrangement:

```
The first floor contains a hydrogen-compatible microchip and a lithium-compatible microchip.
The second floor contains a hydrogen generator.
The third floor contains a lithium generator.
The fourth floor contains nothing relevant.
```

As a diagram (`F#` for a Floor number, `E` for Elevator, `H` for Hydrogen, `L` for Lithium, `M` for Microchip, and `G` for Generator), the initial state looks like this:

```
F4 .  .  .  .  .  
F3 .  .  .  LG .  
F2 .  HG .  .  .  
F1 E  .  HM .  LM 
```

Then, to get everything up to the assembling machine on the fourth floor, the following steps could be taken:

- Bring the Hydrogen-compatible Microchip to the second floor, which is safe because it can get power from the Hydrogen Generator:
```
F4 .  .  .  .  .  
F3 .  .  .  LG .  
F2 E  HG HM .  .  
F1 .  .  .  .  LM 
```
- Bring both Hydrogen-related items to the third floor, which is safe because the Hydrogen-compatible microchip is getting power from its generator:
```
F4 .  .  .  .  .  
F3 E  HG HM LG .  
F2 .  .  .  .  .  
F1 .  .  .  .  LM 
```
- Leave the Hydrogen Generator on floor three, but bring the Hydrogen-compatible Microchip back down with you so you can still use the elevator:
```
F4 .  .  .  .  .  
F3 .  HG .  LG .  
F2 E  .  HM .  .  
F1 .  .  .  .  LM 
```
- At the first floor, grab the Lithium-compatible Microchip, which is safe because Microchips don't affect each other:
```
F4 .  .  .  .  .  
F3 .  HG .  LG .  
F2 .  .  .  .  .  
F1 E  .  HM .  LM 
```
- Bring both Microchips up one floor, where there is nothing to fry them:
```
F4 .  .  .  .  .  
F3 .  HG .  LG .  
F2 E  .  HM .  LM 
F1 .  .  .  .  .  
```
- Bring both Microchips up again to floor three, where they can be temporarily connected to their corresponding generators while the elevator recharges, preventing either of them from being fried:
```
F4 .  .  .  .  .  
F3 E  HG HM LG LM 
F2 .  .  .  .  .  
F1 .  .  .  .  .  
```
- Bring both Microchips to the fourth floor:
```
F4 E  .  HM .  LM 
F3 .  HG .  LG .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
```
- Leave the Lithium-compatible microchip on the fourth floor, but bring the Hydrogen-compatible one so you can still use the elevator; this is safe because although the Lithium Generator is on the destination floor, you can connect Hydrogen-compatible microchip to the Hydrogen Generator there:
```
F4 .  .  .  .  LM 
F3 E  HG HM LG .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
```
- Bring both Generators up to the fourth floor, which is safe because you can connect the Lithium-compatible Microchip to the Lithium Generator upon arrival:
```
F4 E  HG .  LG LM 
F3 .  .  HM .  .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
```
- Bring the Lithium Microchip with you to the third floor so you can use the elevator:
```
F4 .  HG .  LG .  
F3 E  .  HM .  LM 
F2 .  .  .  .  .  
F1 .  .  .  .  .  
```
- Bring both Microchips to the fourth floor:
```
F4 E  HG HM LG LM 
F3 .  .  .  .  .  
F2 .  .  .  .  .  
F1 .  .  .  .  .  
```

In this arrangement, it takes `11` steps to collect all of the objects at the fourth floor for assembly. (Each elevator stop counts as one step, even if nothing is added to or removed from it.)

In your situation, what is the **minimum number of steps** required to bring all of the objects to the fourth floor?

In [45]:
State = namedtuple('State', ['elevator', 'locations'])
Element = namedtuple('Element', ['chip', 'generator'])

chip_re = re.compile("([a-z]+)-compatible microchip")
generator_re = re.compile("([a-z]+) generator")
solved_element = Element(3, 3)

@lru_cache(maxsize=None)
def distance(state):
    return 3 - state.elevator + sum(map(lambda e: 6 - e[0] - e[1], state.locations))

def update_chip(el, delta):
    return Element(el.chip + delta, el.generator)

def update_generator(el, delta):
    return Element(el.chip, el.generator + delta)

def is_valid(state):
    if not (0 <= state.elevator <= 3):
        return False
    for e in state.locations:
        if e.chip == e.generator:
            continue
        for f in state.locations:
            if e != f and e.chip == f.generator:
                return False
    return True

def move(state, i, f_i, j=None, f_j=None):
    for d in [-1, 1]:
        locs = list(state.locations)
        locs[i] = f_i(locs[i], d)
        if j:
            locs[j] = f_j(locs[j], d)
        s = State(state.elevator + d, tuple(sorted(locs)))
        if is_valid(s):
            yield s

def next_states(state):
    for i in range(len(state.locations)):
        if state.locations[i].chip == state.elevator:
            yield from move(state, i, update_chip)
            if state.locations[i].generator == state.elevator:
                yield from move(state, i, update_chip, i, update_generator)
            for j in range(i + 1, len(state.locations)):
                if state.locations[j].chip == state.elevator:
                    yield from move(state, i, update_chip, j, update_chip)
        if state.locations[i].generator == state.elevator:
            yield from move(state, i, update_generator)
            for j in range(i + 1, len(state.locations)):
                if state.locations[j].generator == state.elevator:
                    yield from move(state, i, update_generator, j, update_generator)

start_elements = {}

with Input(11) as f:
    lines = f.readlines()
    for i in range(len(lines)):
        chips = chip_re.findall(lines[i])
        generators = generator_re.findall(lines[i])
        for chip in chips:
            if chip not in start_elements:
                start_elements[chip] = Element(i, None)
            else:
                start_elements[chip] = Element(i, start_elements[chip].generator)
        for generator in generators:
            if generator not in start_elements:
                start_elements[generator] = Element(None, i)
            else:
                start_elements[generator] = Element(start_elements[generator].chip, i)

initial_state = State(0, tuple(sorted(list(start_elements.values()))))
%time print("Day 11.1: {}".format(path_length(next(astar(initial_state, next_states, distance))) - 1))

Day 11.1: 47
CPU times: user 293 ms, sys: 1.64 ms, total: 295 ms
Wall time: 295 ms


### Part 2
You step into the cleanroom separating the lobby from the isolated area and put on the hazmat suit.

Upon entering the isolated containment area, however, you notice some extra parts on the first floor that weren't listed on the record outside:

- An elerium generator.
- An elerium-compatible microchip.
- A dilithium generator.
- A dilithium-compatible microchip.

These work just like the other generators and microchips. You'll have to get them up to assembly as well.

What is the **minimum number of steps** required to bring all of the objects, including these four new ones, to the fourth floor?

In [46]:
start_elements['elerium'] = Element(0, 0)
start_elements['dilithium'] = Element(0, 0)
initial_state = State(0, tuple(sorted(list(start_elements.values()))))
%time print("Day 11.2: {}".format(path_length(next(astar(initial_state, next_states, distance))) - 1))

Day 11.2: 71
CPU times: user 1.84 s, sys: 5.7 ms, total: 1.84 s
Wall time: 1.85 s


## Day 12: Leonardo's Monorail
You finally reach the top floor of this building: a garden with a slanted glass ceiling. Looks like there are no more stars to be had.

While sitting on a nearby bench amidst some tiger lilies, you manage to decrypt some of the files you extracted from the servers downstairs.

According to these documents, Easter Bunny HQ isn't just this building - it's a collection of buildings in the nearby area. They're all connected by a local monorail, and there's another building not far from here! Unfortunately, being night, the monorail is currently not operating.

You remotely connect to the monorail control systems and discover that the boot sequence expects a password. The password-checking logic (your puzzle input) is easy to extract, but the code it uses is strange: it's assembunny code designed for the new computer you just assembled. You'll have to execute the code and get the password.

The assembunny code you've extracted operates on four registers (`a`, `b`, `c`, and `d`) that start at `0` and can hold any integer. However, it seems to make use of only a few instructions:

- `cpy x y` **copies** `x` (either an integer or the value of a register) into register `y`.
- `inc x` **increases** the value of register `x` by one.
- `dec x` **decreases** the value of register `x` by one.
- `jnz x y` **jumps** to an instruction `y` away (positive means forward; negative means backward), but only if `x` is not zero.

The `jnz` instruction moves relative to itself: an offset of `-1` would continue at the previous instruction, while an offset of `2` would **skip over** the next instruction.

For example:
```
cpy 41 a
inc a
inc a
dec a
jnz a 2
dec a
```
The above code would set register `a` to `41`, increase its value by `2`, decrease its value by `1`, and then skip the last `dec a` (because `a` is not zero, so the `jnz a 2` skips it), leaving register `a` at `42`. When you move past the last instruction, the program halts.

After executing the assembunny code in your puzzle input, **what value is left in register `a`**?

In [24]:
class Computer(object):
    def __init__(self, registers={'a': 0, 'b': 0, 'c': 0, 'd': 0}):
        self.registers = registers
        self.pointer = 0

    def value(self, v):
        return self.registers[v] if v in self.registers else int(v)

    def smart_add_from_reg(self, x):
        if len(self.program) > self.pointer + 2:
            next_op, next_args = self.program[self.pointer + 1]
            if next_op == 'dec':
                sec_op, sec_args = self.program[self.pointer + 2]
                if sec_op == 'jnz' and sec_args == (next_args[0], '-2'):
                    self.registers[x] += self.registers[next_args[0]]
                    self.registers[next_args[0]] = 0
                    self.pointer += 2
                    return True
        return False

    def inc(self, x):
        if not self.smart_add_from_reg(x):
            self.registers[x] += 1

    def dec(self, x):
        self.registers[x] -= 1

    def cpy(self, x, y):
        self.registers[y] = self.value(x)

    def jnz(self, x, y):
        if self.value(x):
            self.pointer += self.value(y) - 1  # The last increment is done by the program loop

    def load(self, program):
        """Load the program, split the strings into a tuple (operation, args) where args is another tuple"""
        self.program = [(i[0], tuple(i[1:])) for i in [instruction.strip().split() for instruction in program]]
        return self

    def execute(self, debug=False):
        """Executes a program, which is a list of instructions"""
        while self.pointer < len(self.program):
            op, args = self.program[self.pointer]
            if debug:
                print("{:3}: {:25} {}".format(self.pointer, (op, args), self.registers))
            getattr(self, op)(*args)
            self.pointer += 1
        return self


with open('day12.in', 'r') as f:
    program = f.readlines()

print("Day 12.1: {}".format(Computer().load(program).execute().registers['a']))

Day 12.1: 318083


### Part 2
As you head down the fire escape to the monorail, you notice it didn't start; register `c` needs to be initialized to the position of the ignition key.

If you instead **initialize register `c` to be `1`**, what value is now left in register `a`?

In [25]:
print("Day 12.2: {}".format(Computer({'a': 0, 'b': 0, 'c': 1, 'd': 0}).load(program).execute().registers['a']))

Day 12.2: 9227737


## Day 13: A Maze of Twisty Little Cubicles
You arrive at the first floor of this new building to discover a much less welcoming environment than the shiny atrium of the last one. Instead, you are in a maze of twisty little cubicles, all alike.

Every location in this area is addressed by a pair of non-negative integers (`x,y`). Each such coordinate is either a wall or an open space. You can't move diagonally. The cube maze starts at `0,0` and seems to extend infinitely toward **positive** `x` and `y`; negative values are **invalid**, as they represent a location outside the building. You are in a small waiting area at 1,1.

While it seems chaotic, a nearby morale-boosting poster explains, the layout is actually quite logical. You can determine whether a given `x,y` coordinate will be a wall or an open space using a simple system:

- Find `x*x + 3*x + 2*x*y + y + y*y`.
- Add the office designer's favorite number (your puzzle input).
- Find the binary representation of that sum; count the **number** of bits that are `1`.
  - If the number of bits that are `1` is **even**, it's an **open space**.
  - If the number of bits that are `1` is **odd**, it's a **wall**.

For example, if the office designer's favorite number were `10`, drawing walls as `#` and open spaces as `.`, the corner of the building containing `0,0` would look like this:

```
  0123456789
0 .#.####.##
1 ..#..#...#
2 #....##...
3 ###.#.###.
4 .##..#..#.
5 ..##....#.
6 #...##.###
```

Now, suppose you wanted to reach `7,4`. The shortest route you could take is marked as `O`:

```
  0123456789
0 .#.####.##
1 .O#..#...#
2 #OOO.##...
3 ###O#.###.
4 .##OO#OO#.
5 ..##OOO.#.
6 #...##.###
```

Thus, reaching `7,4` would take a minimum of `11` steps (starting from your current location, `1,1`).

What is the **fewest number of steps required** for you to reach `31,39`?

Your puzzle input is: `1352`

In [26]:
inp = 1352

def is_wall(point):
    x, y = point
    nr = x * x + 3 * x + 2 * x * y + y + y * y
    return bin(nr + inp).count('1') % 2

def open_neighbours(point):
    return [n for n in neighbours4(point) if not is_wall(n) and n[0] >= 0 and n[1] >= 0]

def distance(point):
    return manhattan_distance((31, 39), point)

print("Day 13.1: {}".format(path_length(next(astar((1, 1), open_neighbours, distance)) - 1)))

Day 13.1: 0


### Part 2
**How many locations** (distinct `x,y` coordinates, including your starting location) can you reach in at most `50` steps?

In [27]:
# BFS search which stops at length 50
def bfs_50(initial_state, f_next):
    frontier = [(0, initial_state)]
    seen_states = set([initial_state])
    while frontier:
        path_length, state = heappop(frontier)
        if path_length >= 50:
            return seen_states
        for next_state in f_next(state):
            if next_state in seen_states:
                continue
            heappush(frontier, (path_length + 1, next_state))
            seen_states.add(next_state)

print("Day 13.2: {}".format(len(bfs_50((1, 1), open_neighbours))))

Day 13.2: 135


## Day 14: One-Time Pad
In order to communicate securely with Santa while you're on this mission, you've been using a one-time pad that you generate using a pre-agreed algorithm. Unfortunately, you've run out of keys in your one-time pad, and so you need to generate some more.

To generate keys, you first get a stream of random data by taking the MD5 of a pre-arranged salt (your puzzle input) and an increasing integer index (starting with `0`, and represented in decimal); the resulting MD5 hash should be represented as a string of **lowercase** hexadecimal digits.

However, not all of these MD5 hashes are **keys**, and you need 64 new keys for your one-time pad. A hash is a key **only if**:

- It contains **three** of the same character in a row, like `777`. Only consider the first such triplet in a hash.
- One of the next `1000` hashes in the stream contains that same character **five** times in a row, like `77777`.

Considering future hashes for five-of-a-kind sequences does not cause those hashes to be skipped; instead, regardless of whether the current hash is a key, always resume testing for keys starting with the very next hash.

For example, if the pre-arranged salt is `abc`:

- The first index which produces a triple is `18`, because the MD5 hash of `abc18` contains `...cc38887a5....` However, index 18 does not count as a key for your one-time pad, because none of the next thousand hashes (index `19` through index `1018`) contain `88888`.
- The next index which produces a triple is `39`; the hash of `abc39` contains `eee`. It is also the first key: one of the next thousand hashes (the one at index `816`) contains `eeeee`.
- None of the next six triples are keys, but the one after that, at index `92`, is: it contains `999` and index `200` contains `99999`.
- Eventually, index `22728` meets all of the criteria to generate the `64`th key.

So, using our example salt of `abc`, index `22728` produces the `64`th key.

Given the actual salt in your puzzle input, **what index** produces your `64`th one-time pad key?

Your puzzle input is: `jlmsuwbz`

In [28]:
hasher = md5('jlmsuwbz'.encode('utf-8'))

@lru_cache(maxsize=None)
def pad_func(idx):
    copy = hasher.copy()
    copy.update(str(idx).encode('utf-8'))
    return copy.hexdigest()
    
def one_time_pad(f_pad):
    idx = 0
    while True:
        digest = f_pad(idx)
        for a, b, c in window(digest, 3):
            if a == b == c:
                for i in range(idx + 1, idx + 1000):
                    n = f_pad(i)
                    if 5 * a in n:
                        yield idx
                        found = True
                        break
                break
        idx += 1

print("Day 14.1: {}".format([i for _, i in zip(range(64), one_time_pad(pad_func))][-1]))

Day 14.1: 35186


### Part 2
Of course, in order to make this process even more secure, you've also implemented key stretching.

Key stretching forces attackers to spend more time generating hashes. Unfortunately, it forces everyone else to spend more time, too.

To implement key stretching, whenever you generate a hash, before you use it, you first find the MD5 hash of **that** hash, then the MD5 hash of that hash, and so on, a total of **`2016` additional hashings**. Always use lowercase hexadecimal representations of hashes.

For example, to find the stretched hash for index `0` and salt `abc`:

- Find the MD5 hash of `abc0`: `577571be4de9dcce85a041ba0410f29f`.
- Then, find the MD5 hash of that hash: `eec80a0c92dc8a0777c619d9bb51e910`.
- Then, find the MD5 hash of that hash: `16062ce768787384c81fe17a7a60c7e3`.
- ...repeat many times...
- Then, find the MD5 hash of that hash: `a107ff634856bb300138cac6568c0f24`.

So, the stretched hash for index `0` in this situation is `a107ff....` In the end, you find the original hash (one use of MD5), then find the hash-of-the-previous-hash `2016` times, for a total of `2017` uses of MD5.

The rest of the process remains the same, but now the keys are entirely different. Again for salt `abc`:

- The first triple (`222`, at index `5`) has no matching `22222` in the next thousand hashes.
- The second triple (`eee`, at index `10`) hash a matching `eeeee` at index `89`, and so it is the first key.
- Eventually, index `22551` produces the `64`th key (triple `fff` with matching `fffff` at index `22859`.

Given the actual salt in your puzzle input and using `2016` extra MD5 calls of key stretching, **what index** now produces your `64`th one-time pad key?

In [29]:
@lru_cache(maxsize=None)
def stretch_pad(idx):
    digest = pad_func(idx)
    for _ in range(2016):
        digest = md5(digest.encode('utf-8')).hexdigest()
    return digest

print("Day 14.2: {}".format([i for _, i in zip(range(64), one_time_pad(stretch_pad))][-1]))

Day 14.2: 22429


## Day 15: Timing is Everything
The halls open into an interior plaza containing a large kinetic sculpture. The sculpture is in a sealed enclosure and seems to involve a set of identical spherical capsules that are carried to the top and allowed to bounce through the maze of spinning pieces.

Part of the sculpture is even interactive! When a button is pressed, a capsule is dropped and tries to fall through slots in a set of rotating discs to finally go through a little hole at the bottom and come out of the sculpture. If any of the slots aren't aligned with the capsule as it passes, the capsule bounces off the disc and soars away. You feel compelled to get one of those capsules.

The discs pause their motion each second and come in different sizes; they seem to each have a fixed number of positions at which they stop. You decide to call the position with the slot `0`, and count up for each position it reaches next.

Furthermore, the discs are spaced out so that after you push the button, one second elapses before the first disc is reached, and one second elapses as the capsule passes from one disc to the one below it. So, if you push the button at `time=100`, then the capsule reaches the top disc at `time=101`, the second disc at `time=102`, the third disc at `time=103`, and so on.

The button will only drop a capsule at an integer time - no fractional seconds allowed.

For example, at `time=0`, suppose you see the following arrangement:

```
Disc #1 has 5 positions; at time=0, it is at position 4.
Disc #2 has 2 positions; at time=0, it is at position 1.
```

If you press the button exactly at `time=0`, the capsule would start to fall; it would reach the first disc at `time=1`. Since the first disc was at position `4` at `time=0`, by `time=1` it has ticked one position forward. As a five-position disc, the next position is `0`, and the capsule falls through the slot.

Then, at `time=2`, the capsule reaches the second disc. The second disc has ticked forward two positions at this point: it started at position `1`, then continued to position `0`, and finally ended up at position `1` again. Because there's only a slot at position `0`, the capsule bounces away.

If, however, you wait until `time=5` to push the button, then when the capsule reaches each disc, the first disc will have ticked forward `5+1 = 6` times (to position `0`), and the second disc will have ticked forward `5+2 = 7` times (also to position `0`). In this case, the capsule would fall through the discs and come out of the machine.

However, your situation has more than two discs; you've noted their positions in your puzzle input. What is the **first time you can press the button** to get a capsule?

In [30]:
disc_r = re.compile("Disc #([0-9]+) has ([0-9]+) positions; at time=0, it is at position ([0-9]+).")
discs = []

def disc_as_lambda(size, dist, offset):
    return lambda t: ((t + dist + offset) % size) == 0

def solve(discs):
    s_0, d_0, o_0 = discs[0]
    lambdas = [disc_as_lambda(*disc) for disc in discs]
    t = s_0 - d_0 - o_0
    while True:
        if all(l(t) for l in lambdas):
            return t
        t += s_0

with Input(15) as f:
    for l in f:
        m = disc_r.match(l)
        discs.append((int(m.group(2)), int(m.group(1)), int(m.group(3))))

print("Day 15.1: {}".format(solve(discs)))

Day 15.1: 317371


### Part 2

After getting the first capsule (it contained a star! what great fortune!), the machine detects your success and begins to rearrange itself.

When it's done, the discs are back in their original configuration as if it were `time=0` again, but a new disc with `11` positions and starting at position `0` has appeared exactly one second below the previously-bottom disc.

With this new disc, and counting again starting from `time=0` with the configuration in your puzzle input, what is **the first time you can press the button** to get another capsule?

In [31]:
discs.append((11, len(discs) + 1, 0))
print("Day 15.2: {}".format(solve(discs)))

Day 15.2: 2080951


## Day 16: Dragon Checksum
You're done scanning this part of the network, but you've left traces of your presence. You need to overwrite some disks with random-looking data to cover your tracks and update the local security system with a new checksum for those disks.

For the data to not be suspicious, it needs to have certain properties; purely random data will be detected as tampering. To generate appropriate random data, you'll need to use a modified dragon curve.

Start with an appropriate initial state (your puzzle input). Then, so long as you don't have enough data yet to fill the disk, repeat the following steps:

- Call the data you have at this point "a".
- Make a copy of "a"; call this copy "b".
- Reverse the order of the characters in "b".
- In "b", replace all instances of `0` with `1` and all `1`s with `0`.
- The resulting data is "a", then a single `0`, then "b".

For example, after a single step of this process,

- `1` becomes `100`.
- `0` becomes `001`.
- `11111` becomes `11111000000`.
- `111100001010` becomes `1111000010100101011110000`.

Repeat these steps until you have enough data to fill the desired disk.

Once the data has been generated, you also need to create a checksum of that data. Calculate the checksum **only** for the data that fits on the disk, even if you generated more data than that in the previous step.

The checksum for some given data is created by considering each non-overlapping **pair** of characters in the input data. If the two characters match (`00` or `11`), the next checksum character is a `1`. If the characters do not match (`01` or `10`), the next checksum character is a `0`. This should produce a new string which is exactly half as long as the original. If the length of the checksum is **even**, repeat the process until you end up with a checksum with an **odd** length.

For example, suppose we want to fill a disk of length `12`, and when we finally generate a string of at least length `12`, the first `12` characters are `110010110100`. To generate its checksum:

- Consider each pair: `11`, `00`, `10`, `11`, `01`, `00`.
- These are same, same, different, same, different, same, producing `110101`.
- The resulting string has length `6`, which is **even**, so we repeat the process.
- The pairs are `11` (same), `01` (different), `01` (different).
- This produces the checksum `100`, which has an **odd** length, so we stop.

Therefore, the checksum for `110010110100` is `100`.

Combining all of these steps together, suppose you want to fill a disk of length `20` using an initial state of `10000`:

- Because `10000` is too short, we first use the modified dragon curve to make it longer.
- After one round, it becomes `10000011110` (`11` characters), still too short.
- After two rounds, it becomes `10000011110010000111110` (`23` characters), which is enough.
- Since we only need `20`, but we have `23`, we get rid of all but the first `20` characters: `10000011110010000111`.
- Next, we start calculating the checksum; after one round, we have `0111110101`, which `10` characters long (**even**), so we continue.
- After two rounds, we have `01100`, which is `5` characters long (**odd**), so we are done.

In this example, the correct checksum would therefore be `01100`.

The first disk you have to fill has length `272`. Using the initial state in your puzzle input, **what is the correct checksum**?

Your puzzle input is: `01000100010010111`

In [32]:
def dragon_curve(initial_state, disk_size):
    arr = bitarray(initial_state)
    while len(arr) < disk_size:
        inv = arr.copy()
        inv.reverse()
        inv.invert()
        arr.append(0)
        arr.extend(inv)
        
    arr = arr[:disk_size]
    while len(arr) % 2 == 0:
        arr = [1 if a == b else 0 for a, b in zip(arr[::2], arr[1::2])]

    return ''.join(['0' if not c else '1' for c in arr])

print("Day 16.1: {}".format(dragon_curve("01000100010010111", 272)))

Day 16.1: 10010010110011010


### Part 2
The second disk you have to fill has length `35651584`. Again using the initial state in your puzzle input, **what is the correct checksum** for this disk?

In [42]:
%time print("Day 16.2: {}".format(dragon_curve("01000100010010111", 35651584)))

Day 16.2: 01010100101011100
CPU times: user 2.97 s, sys: 333 ms, total: 3.3 s
Wall time: 3.31 s


## Day 17: Two Steps Forward
You're trying to access a secure vault protected by a `4x4` grid of small rooms connected by doors. You start in the top-left room (marked `S`), and you can access the vault (marked `V`) once you reach the bottom-right room:

```
#########
#S| | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | | #
#-#-#-#-#
# | | |  
####### V
```

Fixed walls are marked with `#`, and doors are marked with `-` or `|`.

The doors in your **current room** are either open or closed (and locked) based on the hexadecimal MD5 hash of a passcode (your puzzle input) followed by a sequence of uppercase characters representing the **path you have taken so far** (`U` for up, `D` for down, `L` for left, and `R` for right).

Only the first four characters of the hash are used; they represent, respectively, the doors **up, down, left, and right** from your current position. Any `b`, `c`, `d`, `e`, or `f` means that the corresponding door is **open**; any other character (any number or `a`) means that the corresponding door is **closed and locked**.

To access the vault, all you need to do is reach the bottom-right room; reaching this room opens the vault and all doors in the maze.

For example, suppose the passcode is `hijkl`. Initially, you have taken no steps, and so your path is empty: you simply find the MD5 hash of `hijkl` alone. The first four characters of this hash are `ced9`, which indicate that up is open (`c`), down is open (`e`), left is open (`d`), and right is closed and locked (`9`). Because you start in the top-left corner, there are no "up" or "left" doors to be open, so your only choice is **down**.

Next, having gone only one step (down, or `D`), you find the hash of `hijklD`. This produces `f2bc`, which indicates that you can go back up, left (but that's a wall), or right. Going right means hashing `hijklDR` to get `5745` - all doors closed and locked. However, going **up** instead is worthwhile: even though it returns you to the room you started in, your path would then be `DU`, opening a different set of doors.

After going `DU` (and then hashing `hijklDU` to get `528e`), only the right door is open; after going `DUR`, all doors lock. (Fortunately, your actual passcode is not `hijkl`).

Passcodes actually used by Easter Bunny Vault Security do allow access to the vault if you know the right path. For example:

- If your passcode were `ihgpwlah`, the shortest path would be `DDRRRD`.
- With `kglvqrro`, the shortest path would be `DDUDRLRRUDRD`.
- With `ulqzkmiv`, the shortest would be `DRURDRUDDLLDLUURRDULRLDUUDDDRR`.

Given your vault's passcode, **what is the shortest path** (the actual path, not just the length) to reach the vault?

In [34]:
def dist_from_vault(state):
    return manhattan_distance(state[0], (3, 3))

def paths(state):
    point, m, hasher = state
    possibles = [int(c, 16) for c in hasher.hexdigest()[:4]]
    if possibles[0] > 10 and point[1] > 0:
        h = hasher.copy()
        h.update('U'.encode('utf-8'))
        yield ((point[0], point[1] - 1), m + 'U', h)
    if possibles[1] > 10 and point[1] < 3:
        h = hasher.copy()
        h.update('D'.encode('utf-8'))
        yield ((point[0], point[1] + 1), m + 'D', h)
    if possibles[2] > 10 and point[0] > 0:
        h = hasher.copy()
        h.update('L'.encode('utf-8'))
        yield ((point[0] - 1, point[1]), m + 'L', h)
    if possibles[3] > 10 and point[0] < 3:
        h = hasher.copy()
        h.update('R'.encode('utf-8'))
        yield ((point[0] + 1, point[1]), m + 'R', h)

initial_state = ((0, 0), '', md5("awrkjxxr".encode('utf-8')))
print("Day 17.1: {}".format(next(next(astar(initial_state, paths, dist_from_vault)))[1]))

Day 17.1: RDURRDDLRD


### Part 2
You're curious how robust this security solution really is, and so you decide to find longer and longer paths which still provide access to the vault. You remember that paths always end the first time they reach the bottom-right room (that is, they can never pass through it, only end in it).

For example:

- If your passcode were `ihgpwlah`, the longest path would take `370` steps.
- With `kglvqrro`, the longest path would be `492` steps long.
- With `ulqzkmiv`, the longest path would be `830` steps long.

What is the **length of the longest path** that reaches the vault?

In [35]:
print("Day 17.2: {}".format(len(next([p for p in astar(initial_state, paths, dist_from_vault)][-1])[1])))

Day 17.2: 526


## Day 17: Like a Rogue
As you enter this room, you hear a loud click! Some of the tiles in the floor here seem to be pressure plates for traps, and the trap you just triggered has run out of... whatever it tried to do to you. You doubt you'll be so lucky next time.

Upon closer examination, the traps and safe tiles in this room seem to follow a pattern. The tiles are arranged into rows that are all the same width; you take note of the safe tiles (`.`) and traps (`^`) in the first row (your puzzle input).

The type of tile (trapped or safe) in each row is based on the types of the tiles in the same position, and to either side of that position, in the previous row. (If either side is off either end of the row, it counts as "safe" because there isn't a trap embedded in the wall.)

For example, suppose you know the first row (with tiles marked by letters) and want to determine the next row (with tiles marked by numbers):

```
ABCDE
12345
```

The type of tile `2` is based on the types of tiles `A`, `B`, and `C`; the type of tile `5` is based on tiles D, E, and an imaginary "safe" tile. Let's call these three tiles from the previous row the **left**, **center**, and **right** tiles, respectively. Then, a new tile is a trap only in one of the following situations:

- Its **left** and **center** tiles are traps, but its **right** tile is not.
- Its **center** and **right** tiles are traps, but its **left** tile is not.
- Only its **left** tile is a trap.
- Only its **right** tile is a trap.

In any other situation, the new tile is safe.

Then, starting with the row `..^^.`, you can determine the next row by applying those rules to each new tile:

- The leftmost character on the next row considers the left (nonexistent, so we assume "safe"), center (the first `.`, which means "safe"), and right (the second `.`, also "safe") tiles on the previous row. Because all of the trap rules require a trap in at least one of the previous three tiles, the first tile on this new row is also safe, `.`.
- The second character on the next row considers its left (`.`), center (`.`), and right (`^`) tiles from the previous row. This matches the fourth rule: only the right tile is a trap. Therefore, the next tile in this new row is a trap, `^`.
- The third character considers `.^^`, which matches the second trap rule: its center and right tiles are traps, but its left tile is not. Therefore, this tile is also a trap, `^`.
- The last two characters in this new row match the first and third rules, respectively, and so they are both also traps, `^`.

After these steps, we now know the next row of tiles in the room: `.^^^^.` Then, we continue on to the next row, using the same rules, and get `^^..^.` After determining two new rows, our map looks like this:

```
..^^.
.^^^^
^^..^
```
Here's a larger example with ten tiles per row and ten rows:

```
.^^.^.^^^^
^^^...^..^
^.^^.^.^^.
..^^...^^^
.^^^^.^^.^
^^..^.^^..
^^^^..^^^.
^..^^^^.^^
.^^^..^.^^
^^.^^^..^^
```
In ten rows, this larger example has `38` safe tiles.

Starting with the map in your puzzle input, in a total of `40` rows (including the starting row), **how many safe tiles** are there?

In [36]:
def trap_room_row(row):
    while True:
        yield row
        row = ['^' if left != right else '.' for left, _, right in window(['.'] + row + ['.'], 3)]
        
first_row = list(".^^^.^.^^^^^..^^^..^..^..^^..^.^.^.^^.^^....^.^...^.^^.^^.^^..^^..^.^..^^^.^^...^...^^....^^.^^^^^^^")
print("Day 18.1: {}".format(sum(map(lambda t: t[1].count('.'), zip(range(40), trap_room_row(first_row))))))

Day 18.1: 1939


### Part 2
**How many safe tiles** are there in a total of `400000` rows?

In [37]:
%time print("Day 18.2: {}".format(sum(map(lambda t: t[1].count('.'), zip(range(400000), trap_room_row(first_row))))))

Day 18.2: 19999535
CPU times: user 11.3 s, sys: 20.7 ms, total: 11.3 s
Wall time: 11.3 s


## Day 19: An Elephant Named Joseph
The Elves contact you over a highly secure emergency channel. Back at the North Pole, the Elves are busy misunderstanding White Elephant parties.

Each Elf brings a present. They all sit in a circle, numbered starting with position `1`. Then, starting with the first Elf, they take turns stealing all the presents from the Elf to their left. An Elf with no presents is removed from the circle and does not take turns.

For example, with five Elves (numbered `1` to `5`):

```
  1
5   2
 4 3
```

- Elf `1` takes Elf `2`'s present.
- Elf `2` has no presents and is skipped.
- Elf `3` takes Elf 4's present.
- Elf `4` has no presents and is also skipped.
- Elf `5` takes Elf `1`'s two presents.
- Neither Elf `1` nor Elf `2` have any presents, so both are skipped.
- Elf `3` takes Elf `5`'s three presents.

So, with **five** Elves, the Elf that sits starting in position `3` gets all the presents.

With the number of Elves given in your puzzle input, **which Elf gets all the presents**?

Your puzzle input is: `3004953`

In [38]:
def elves_1(elves):
    elves_left = range(1, elves + 1)
    while len(elves_left) > 1:
        elves_left = elves_left[::2] if len(elves_left) % 2 == 0 else elves_left[2::2]
    return elves_left[0]

print("Day 19.1: {}".format(elves_1(3004953)))

Day 19.1: 1815603


### Part 2
Realizing the folly of their present-exchange rules, the Elves agree to instead steal presents from the Elf **directly across the circle**. If two Elves are across the circle, the one on the left (from the perspective of the stealer) is stolen from. The other rules remain unchanged: Elves with no presents are removed from the circle entirely, and the other elves move in slightly to keep the circle evenly spaced.

For example, with five Elves (again numbered `1` to `5`):

- The Elves sit in a circle; Elf `1` goes first:
```
  1
5   2
 4 3
```
- Elves `3` and `4` are across the circle; Elf `3`'s present is stolen, being the one to the left. Elf `3` leaves the circle, and the rest of the Elves move in:
```
  1           1
5   2  -->  5   2
 4 -          4
```
- Elf `2` steals from the Elf directly across the circle, Elf `5`:
```
  1         1 
-   2  -->     2
  4         4 
```
- Next is Elf `4` who, choosing between Elves `1` and `2`, steals from Elf `1`:
```
 -          2  
    2  -->
 4          4
```
- Finally, Elf `2` steals from Elf `4`:
```
 2
    -->  2  
 -
```
So, with **five** Elves, the Elf that sits starting in position 2 gets all the presents.

With the number of Elves given in your puzzle input, **which Elf now gets all the presents**?

In [39]:
class Elf(object):
    def __init__(self, value):
        self.value = value
        self.prev = None
        self.next = None

    def delete(self):
        self.prev.next = self.next
        self.next.prev = self.prev

    def __repr__(self):
        return "Elf[{}]".format(self.value)

def elves_2(nr_elves):
    elves = list(map(Elf, range(1, nr_elves + 1)))
    for i in range(nr_elves):
        elves[i].next = elves[(i + 1) % nr_elves]
        elves[i].prev = elves[(i - 1) % nr_elves]

    curr_elf, steal = elves[0], elves[int(math.floor(nr_elves / 2))]
    for i in range(nr_elves - 1):
        steal.delete()
        steal = steal.next
        if nr_elves % 2:
            steal = steal.next
        curr_elf = curr_elf.next
        nr_elves -= 1

    return curr_elf.value

print("Day 19.2: {}".format(elves_2(3004953)))

Day 19.2: 1410630


## Day 20: Firewall Rules
You'd like to set up a small hidden computer here so you can use it to get back into the network later. However, the corporate firewall only allows communication with certain external IP addresses.

You've retrieved the list of blocked IPs from the firewall, but the list seems to be messy and poorly maintained, and it's not clear which IPs are allowed. Also, rather than being written in dot-decimal notation, they are written as plain 32-bit integers, which can have any value from `0` through `4294967295`, inclusive.

For example, suppose only the values `0` through `9` were valid, and that you retrieved the following blacklist:

```
5-8
0-2
4-7
```
The blacklist specifies ranges of IPs (inclusive of both the start and end value) that are **not** allowed. Then, the only IPs that this firewall allows are `3` and `9`, since those are the only numbers not in any range.

Given the list of blocked IPs you retrieved from the firewall (your puzzle input), **what is the lowest-valued IP** that is not blocked?

In [40]:
def find_lowest_ip(firewall_rules):
    current_ip = 0
    for low, high in firewall_rules:
        if current_ip < low:
            return current_ip
        elif current_ip < high:
            current_ip = high + 1
    return current_ip

with Input(20) as f:
    firewall_rules = [tuple(map(int, line.strip().split('-'))) for line in f]
    firewall_rules.sort()
    print("Day 20.1: {}".format(find_lowest_ip(firewall_rules)))

Day 20.1: 4793564


### Part 2
**How many IPs** are allowed by the blacklist?

In [41]:
def count_ips(firewall_rules):
    current_ip = 0
    for low, high in firewall_rules:
        if current_ip < low:
            yield low - current_ip
        if current_ip < high:
            current_ip = high + 1
    yield max(0, 4294967295 - current_ip)

print("Day 20.2: {}".format(sum(count_ips(firewall_rules))))

Day 20.2: 146


## Day 21: Scrambled Letters and Hash
TODO

## Day 22: Grid Computing
You gain access to a massive storage cluster arranged in a grid; each storage node is only connected to the four nodes directly adjacent to it (three if the node is on an edge, two if it's in a corner).

You can directly access data **only** on node `/dev/grid/node-x0-y0`, but you can perform some limited actions on the other nodes:

You can get the disk usage of all nodes (via df). The result of doing this is in your puzzle input.
You can instruct a node to **move** (not copy) **all** of its data to an adjacent node (if the destination node has enough space to receive the data). The sending node is left empty after this operation.
Nodes are named by their position: the node named `node-x10-y10` is adjacent to nodes `node-x9-y10`, `node-x11-y10`, `node-x10-y9`, and `node-x10-y11`.

Before you begin, you need to understand the arrangement of data on these nodes. Even though you can only move data between directly connected nodes, you're going to need to rearrange a lot of the data to get access to the data you need. Therefore, you need to work out how you might be able to shift data around.

To do this, you'd like to count the number of **viable pairs** of nodes. A viable pair is any two nodes (A,B), **regardless of whether they are directly connected**, such that:

- Node A is **not** empty (its Used is not zero).
- Nodes A and B are **not the same** node.
- The data on node A (its Used) **would fit** on node B (its Avail).

**How many viable pairs** of nodes are there?